# ✅ Proyecto Final

### Análisis de Datos de Emprendedores en la Provincia de Entre Ríos (2016-2019)

### CoderHouse: Data Science III - NLP & Deep Learning aplicado a Ciencia de Datos

### Rodrigo Sosa - Cohorte 2024/2025

**LinkedIn**: [https://www.linkedin.com/in/sosarodrigo/](https://www.linkedin.com/in/sosarodrigo/)

![Portada](https://github.com/sosarodrigox/coderhouse_emprendedores_it/blob/main/DataBanner2.png?raw=1)

📚 **Introducción**

El dataset utilizado en este análisis proviene de un archivo de Google Sheets asociado al formulario de Google desarrollado para la **Línea de Acción "Incorporación de Tecnología"**, un programa implementado por el Ministerio de Desarrollo Social de Entre Ríos entre marzo de 2016 y agosto de 2019. Este formulario recopiló información clave de emprendedores interesados en recibir apoyo financiero para la adquisición de herramientas y equipamiento.

✨ **Proyectos Previos**
En la etapa Data Science II de la Carrera de Ciencia de Datos, realicé dos proyectos relacionados:

1. **EDA y Data Wrangling**:
[EDA y Data Wrangling Notebook](./DataWrangling_EDA_SosaRodrigo(Corregido).ipynb)
   - Objetivo: Explorar y limpiar los datos para identificar patrones relacionados con las aprobaciones.
   - Principales hallazgos: Factores como dedicación semanal, sector productivo y participación en programas de apoyo estaban fuertemente correlacionados con las aprobaciones.

2. **Machine Learning**: 
[Machine Learning Notebook](./MachineLearning_SosaRodrigo.ipynb)
   - Objetivo: Desarrollar un modelo predictivo para predecir la aprobación de proyectos.
   - Principal desafío: Un desbalance significativo en las clases (1,174 aprobados vs. 75 no aprobados), lo que dificultó la correcta identificación de los casos no aprobados.
   - Resultados: Aunque se aplicaron técnicas como SMOTE y modelos avanzados como XGBoost, el desbalanceo de datos limitó la capacidad predictiva del modelo para identificar proyectos no aprobados.

<p align="center">
   <img src="img/aprobados.png" alt="Distribución de aprobados y no aprobados">
</p>

★ **Enfoque del Proyecto Actual**

En este proyecto, se propone explorar un enfoque basado en **Procesamiento de Lenguaje Natural (NLP)**, utilizando las observaciones y conclusiones registradas por el equipo técnico en la columna *"Evaluación Final"*. Este texto contiene descripciones detalladas que podrían influir en el resultado de aprobación o desaprobación de los emprendedores.

✨ **Objetivo Principal**
Desarrollar un modelo predictivo que clasifique los proyectos como "aprobados" o "no aprobados" a partir del análisis de texto en las evaluaciones. Esto se logrará mediante técnicas de NLP para procesar y vectorizar el texto, combinado con algoritmos de clasificación supervisada.

✨ **Estrategia Metodológica**

1. **Preprocesamiento del Texto:**
   - Tokenización, lematización y remoción de *stopwords*.
   - Análisis de frecuencia y creación de representaciones vectoriales (TF-IDF).

2. **Análisis de Sentimientos:**
   - Evaluar la polaridad y el sentimiento general del texto como una característica adicional.

3. **Modelo Predictivo:**
   - Entrenar un modelo supervisado basado en las representaciones vectoriales del texto y evaluar su desempeño con métricas clave como Recall, F1-score y AUC-ROC.

✨ **Relevancia**
Este análisis tiene como objetivo aportar una nueva perspectiva al proceso de selección de proyectos, integrando herramientas de NLP para comprender mejor el impacto del lenguaje utilizado en las evaluaciones técnicas. Además, busca superar las limitaciones de los modelos de Machine Learning tradicionales al incorporar información contextual valiosa directamente desde los datos textuales.

Como **Técnico en Gestión y Administración Pública (UNER)** y **Técnico en Ciencia de Datos (ISPC)**, mi interés radica en aplicar técnicas avanzadas de ciencia de datos para mejorar la eficiencia de las políticas públicas. Este proyecto representa una oportunidad para explorar nuevos enfoques en la toma de decisiones, optimizando los recursos disponibles y apoyando el desarrollo de los emprendedores locales. 📊

In [1]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    roc_auc_score,
    accuracy_score,
)
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import plotly.express as px
from imblearn.over_sampling import SMOTE

# Descargar recursos de NLTK
nltk.download("punkt")
nltk.download("stopwords")
nltk.download("wordnet")

[nltk_data] Downloading package punkt to /Users/negrux/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/negrux/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/negrux/nltk_data...


True

In [2]:
# Ruta del archivo Excel en GitHub
url = "https://github.com/sosarodrigox/coderhouse_emprendedores_it/raw/main/emprendedores%20_it.xlsx"

df = pd.read_excel(url)

# Configuración para mostrar todas las columnas
pd.set_option("display.max_columns", None)
pd.set_option("display.expand_frame_repr", False)

df.head(2)

,Marca temporal,Marcar entrevista como:,Fecha de la entrevista:,Fecha de la entrevista-Final:,Equipo Técnico:,Referencia a Expediente:,Localidad:,Departamento:,Fecha de Nacimiento-Final:,Sexo:,Estado Civil:,Nivel de Estudio Actual:,Situación Laboral PRINCIPAL:,Situación Habitacional:,Tipo de vivienda:,Situación Familiar:,Situación ante el IVA:,Observaciones Respecto al IVA:,Tramitó la Exención Impositiva en ATER?:,Ingreso MENSUAL de su Economía Familiar:,Nombre del emprendimiento:,Facebook del emprendimiento:,E-mail (Del Emprendimiento):,Descripción del Emprendimiento:,Antigüedad del Emprendimiento (Años):,Antigüedad del Emprendimiento (Meses):,Tiempo que dedica al emprendimiento (Días a la semana):,Tiempo que dedica al emprendimiento (Horas por día):,Realizó algún tipo de capacitación previa para su emprendimiento:,"En caso de haber recibido algún tipo de capacitación, marque cuales:",Cantidad de personas involucradas en el emprendimiento:,Qué función cumplen?:,Cómo realiza sus ventas? / Cómo ofrece sus servicios?:,Utiliza tarjeta de crédito para realizar sus ventas?:,Cantidad de clientes:,Facturación Mensual (Ingresos Brutos):,Costo Mensual (Egresos):,Ganancia Mensual-Final:,Detallar el nivel de producción semanal:,Donde compra la materia prima?:,El emprendimiento se desarrolla en un ámbito:,Donde realiza su producción? / Donde ofrece su servicio?:,Condiciones del espacio físico donde desarrolla el emprendimiento:,Observaciones del espacio productivo:,Qué habilitaciones posee su espacio productivo?:,Servicio Eléctrico:,Servicio de Gas:,Participó en alguno de los siguientes programas del Min. de Desarrollo Social de E.R.?:,En qué invirtió o para qué utilizó el fortalecimiento adquirido?:,Posee otras fuentes de financiamiento que haya destinado a su emprendimiento?:,Equipamiento con el que cuenta:,Herramientas o equipamiento que solicita:,Aporte que obtendría en su emprendimiento a partir de la incorporación de la/las herramienta/as solicitada/as:,Devolución con respecto a la gestión de la herramienta o equipamiento:,Evaluación Final:,Herramienta/as aprobada/as por Equipo Técnico:,Monto Máximo ($) aprobado por Equipo Técnico:,Herramienta comprada Efectivamente:,Observaciones de la entrevista:,Rubro:,Sub-Rubro:,Sub-Rubro:.1,Sub-Rubro-Final:,Actividad:,Actividad:.1,Actividad:.2,Actividad:.3,Actividad:.4,Actividad:.5,Actividad:.6,Actividad:.7,Actividad:.8,Actividad:.9,Actividad:.10,Actividad:.11,Actividad:.12,Actividad:.13,Actividad:.14,Actividad:.15,Actividad-Final:
0,2016-03-18 09:27:31,La entrevista se realizó correctamente.,2016-03-17 00:00:00,2016-03-17 00:00:00,Gisela,NaN,Gualeguaychú,Gualeguaychú,1971-02-09 00:00:00,Femenino,Casada/o,Secundario Completo,Autoempleado,Vivienda Alquilada,TIPO B,Vive con su esposo quien realiza changas de ja...,Monotributo Social,NaN,SI,6000.0,Lo Nuestro,Lo Nuestro,NaN,"Pre pizzas, pasta frolas, cremonas, pan sabori...",12.0,1.0,5.0,4.0,SI,Capacitación en el Oficio,1,"Los dos hacen todo, Brenda hace lo de reposter...","Tiene un puesto fijo en una Feria., Coordina l...",NO,15,4000.0,2500.0,NaN,10 a 30 kg de harina,mayoristas locales,Urbano,En su casa. (No posee un espacio destinado al ...,Regular.,"Muy chico. 3 x 5 m. Techo cieloraso, piso cera...",Ninguna,Instalación Monofásica.,Gas Envasado.,Programa de Microcréditos para el Desarrollo d...,Insumos- Elementos de Cocina,No,"Horno 6 moldes pizzero, Heladera familiar, bat...",Amasadora y sobadora de mesa,Para agregar otros productos con hojaldre y ag...,Aprobado,Se considera necesario otorgar la maquinaria s...,Amasadora y Sobadora,22.0,NaN,La producción es por pedido pero tienen pedido...,Producción,NaN,Panificación,Panificación,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pastelería,NaN,NaN,NaN,Pastelería
1,2016-03-31 11:41:15,La entrevista se realizó correctamente.,2016-03-18 00:00:00,2016-03-18 00:00:00,Rodrigo,NaN,Victoria,Victoria,1971-02-10 00:00:00,Masculino,Casada/o,Secundario Incompleto,Empleado,Vivienda Propia,Tipo A,El emprendedor